# Cohere and Azure with LangChain

The following notebook gives examples of how to call Cohere's models deployed on Azure with LangChain

First you will need to install the Cohere's package in LangChain.

In [ ]:
# %pip install langchain_cohere

Then to set up the Cohere client you will need to input the Azure endpoint URL and token.

In [ ]:
import cohere
import getpass
from langchain_cohere import ChatCohere
from langchain_core.messages import HumanMessage
import os

os.environ["COHERE_API_KEY"] = getpass.getpass(prompt="Azure AD token:")
url = getpass.getpass(
    prompt="Azure endpoint url this should be without /chat or /embed and with /v1 at the end:"
)

co = ChatCohere(base_url=url)

## Chat

In [ ]:
messages = [HumanMessage(content="Hello")]
response = co.invoke(messages)
print(response.content)

In [ ]:
messages = [
    HumanMessage(
        content="How can organizations effectively manage and secure their cloud infrastructure?"
    )
]
for chunk in co.stream(messages):
    print(chunk.content, end="", flush=True)

## Chat with RAG

Using LangChain's Wiki Retriever. You will need the wikipedia and langchain packages for this example.

In [ ]:
%pip install --upgrade --quiet wikipedia
%pip install --upgrade --quiet langchain

In [ ]:
from langchain_core.messages import AIMessage
from langchain.retrievers import WikipediaRetriever
from typing import Dict, List, Any
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate

prompt = lambda x: [
    HumanMessage(
        x["question"], additional_kwargs={"documents": x.get("documents", None)}
    )
]

rag_x_citations_chain = (
    {"documents": WikipediaRetriever(), "question": RunnablePassthrough()}
    | RunnablePassthrough()
    | prompt
    | co
)
response = rag_x_citations_chain.invoke("What is Task Decomposition?")
print(response.content)
print(response.additional_kwargs["citations"])
print(response.additional_kwargs["documents"])

## Chat with Tools


In [ ]:
%pip install --quiet langchain langchain-core langchain-cohere

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_cohere import ChatCohere, create_cohere_tools_agent
from langchain.agents import AgentExecutor
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_core.messages import HumanMessage
from langchain_core.pydantic_v1 import BaseModel, Field
import os
import getpass

os.environ["TAVILY_API_KEY"] = getpass.getpass(prompt="Tavily API key:")

internet_search = TavilySearchResults()
internet_search.name = "internet_search"
internet_search.description = "Returns a list of relevant document snippets for a textual query retrieved from the internet."


class TavilySearchInput(BaseModel):
    query: str = Field(description="Query to search the internet with")


internet_search.args_schema = TavilySearchInput

prompt = ChatPromptTemplate.from_messages(
    [
        HumanMessage(
            content="In what year was the company that was founded as Sound of Music added to the S&P 500?"
        )
    ]
)

llm = co.bind()
agent = create_cohere_tools_agent(llm, [internet_search], prompt)
agent_executor = AgentExecutor(agent=agent, tools=[internet_search], verbose=True)
agent_executor.invoke({})

## Embed

In [ ]:
import cohere
import getpass
from langchain_cohere import CohereEmbeddings
from langchain_core.messages import HumanMessage
import os

# os.environ["COHERE_API_KEY"] = getpass.getpass(prompt="Azure AD token:")
url = getpass.getpass(
    prompt="Azure endpoint url this should be without /chat or /embed and with /v1 at the end:"
)

co = CohereEmbeddings(base_url=url)
query_result = co.embed(
    texts=["Embed this sentence please"], input_type="classification"
)
print(query_result)

# Rerank

In [ ]:
import cohere
import getpass
from langchain_cohere import CohereRerank
import os

os.environ["COHERE_API_KEY"] = getpass.getpass(prompt="Azure AD token:")

co = CohereRerank(cohere_api_key=os.environ["COHERE_API_KEY"])
query = "What is the capital of the United States?"
docs = [
    "Carson City is the capital city of the American state of Nevada. At the 2010 United States Census, Carson City had a population of 55,274.",
    "The Commonwealth of the Northern Mariana Islands is a group of islands in the Pacific Ocean that are a political division controlled by the United States. Its capital is Saipan.",
    "Charlotte Amalie is the capital and largest city of the United States Virgin Islands. It has about 20,000 people. The city is on the island of Saint Thomas.",
    "Washington, D.C. (also known as simply Washington or D.C., and officially as the District of Columbia) is the capital of the United States. It is a federal district. The President of the USA and many major national government offices are in the territory. This makes it the political center of the United States of America.",
    "Capital punishment (the death penalty) has existed in the United States since before the United States was a country. As of 2017, capital punishment is legal in 30 of the 50 states. The federal government (including the United States military) also uses capital punishment.",
]
results = co.rerank(model="rerank-english-v3.0", query=query, documents=docs, top_n=5)
results